# 4. Comparing training runs and Hyperparameter (HP) tuning

In this chapter, you will direct your attention towards the analysis of model performance and the fine-tuning of hyperparameters. You will acquire practical expertise in comparing metrics and visualizations across different branches to assess changes in model performance. You will conduct hyperparameter tuning using scikit-learn's GridSearchCV. Furthermore, you will delve into the automation of pull requests using the optimal model configuration.

## 4.1 Comparing metrics and plots in DVC

### Running the process manually

In [1]:
!python .\ml-example3\preprocess_data.py
!python .\ml-example3\train.py

====================Test Set Metrics==================
{
  "accuracy": 0.947,
  "precision": 0.988,
  "recall": 0.7702,
  "f1_score": 0.8656
}


### Preparing the environment

In [2]:
# Removing files from git versioning
# To ensure this files can be tracked in dvc.
!git add ml-example3\processed-data\weather.csv
!git add ml-example3\evaluation-result\confusion_matrix.png
!git commit -m "tracking ml-ex3 .. weather.cs and confusion_matrix.png"
!git rm -r --cached ml-example3\processed-data\weather.csv
!git rm -r --cached ml-example3\evaluation-result\confusion_matrix.png
!git commit -m "stop tracking ml-ex3 .. weather.cs and confusion_matrix.png"

The following paths are ignored by one of your .gitignore files:
ml-example3/processed-data/weather.csv
hint: Use -f if you really want to add them.
hint: Disable this message with "git config advice.addIgnoredFile false"
The following paths are ignored by one of your .gitignore files:
ml-example3/evaluation-result/confusion_matrix.png
hint: Use -f if you really want to add them.
hint: Disable this message with "git config advice.addIgnoredFile false"


On branch myfeature
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Track4-MLTrainning.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


fatal: pathspec 'ml-example3\processed-data\weather.csv' did not match any files
fatal: pathspec 'ml-example3\evaluation-result\confusion_matrix.png' did not match any files


On branch myfeature
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Track4-MLTrainning.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [3]:
# Removing any previous `dvc.yaml` file
# Removing any existing previous yml file
!del dvc.yaml

### Working with DVC pipeline for `ml-example3`

In [4]:
# Prepare the yaml file
yml_str = f"""
stages:
  preprocess:
    cmd: python ml-example3/preprocess_data.py
    deps:
    - ml-example3/raw-data/weather.csv
    - ml-example3/preprocess_data.py
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/processed-data/weather.csv
  train:
    cmd: python ml-example3/train.py
    deps:
    - ml-example3/metrics_and_plots.py
    - ml-example3/model.py
    - ml-example3/processed-data/weather.csv
    - ml-example3/train.py
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/evaluation-result/confusion_matrix.png
    metrics:
      - ml-example3/evaluation-result/metrics.json:
          cache: false
    plots:
      - ml-example3/evaluation-result/predictions.csv:
          template: confusion_normalized
          x: predicted_label
          y: true_label
          x_label: 'Predicted label'
          y_label: 'True label'
          title: Confusion matrix
          cache: false
"""
with open("dvc.yaml", "w") as f:
    f.write(yml_str)

In [5]:
# Reviewing the dvc.yaml file
!more dvc.yaml


stages:
  preprocess:
    cmd: python ml-example3/preprocess_data.py
    deps:
    - ml-example3/raw-data/weather.csv
    - ml-example3/preprocess_data.py
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/processed-data/weather.csv
  train:
    cmd: python ml-example3/train.py
    deps:
    - ml-example3/metrics_and_plots.py
    - ml-example3/model.py
    - ml-example3/processed-data/weather.csv
    - ml-example3/train.py
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/evaluation-result/confusion_matrix.png
    metrics:
      - ml-example3/evaluation-result/metrics.json:
          cache: false
    plots:
      - ml-example3/evaluation-result/predictions.csv:
          template: confusion_normalized
          x: predicted_label
          y: true_label
          x_label: 'Predicted label'
          y_label: 'True label'
          title: Confusion matrix
          cache: false


In [6]:
# Reviewing the process
!dvc dag

+------------+ 
| preprocess | 
+------------+ 
       *       
       *       
       *       
  +-------+    
  | train |    
  +-------+    


In [7]:
# Reviewing the comands that the pipeline will execute:
!dvc repro --dry

Stage 'preprocess' didn't change, skipping
Stage 'train' didn't change, skipping
Data and pipelines are up to date.


In [8]:
!dvc repro

Stage 'preprocess' didn't change, skipping
Stage 'train' didn't change, skipping
Data and pipelines are up to date.


### Querying and comparing DVC metrics

In [9]:
!dvc metrics show

Path                                        accuracy    f1_score    precision    recall
ml-example3\evaluation-result\metrics.json  0.947       0.8656      0.988        0.7702


In [13]:
!dvc metrics show --md

| Path                                       | accuracy   | f1_score   | precision   | recall   |
|--------------------------------------------|------------|------------|-------------|----------|
| ml-example3\evaluation-result\metrics.json | 0.947      | 0.8656     | 0.988       | 0.7702   |



In [17]:
!dvc metrics diff --all

Path                                        Metric     HEAD    workspace    Change
ml-example3\evaluation-result\metrics.json  accuracy   0.947   0.947        0.0
ml-example3\evaluation-result\metrics.json  f1_score   0.8656  0.8656       0.0
ml-example3\evaluation-result\metrics.json  precision  0.988   0.988        0.0
ml-example3\evaluation-result\metrics.json  recall     0.7702  0.7702       0.0


In [14]:
# Compare metrics with main branch
!git fetch --prune
!dvc metrics diff master

From https://github.com/jacesca/CICD-Workflow
   f9013d6..d34d883  master     -> origin/master
   966be62..1512f7c  myfeature  -> origin/myfeature


Path                                        Metric     master    workspace    Change
ml-example3\evaluation-result\metrics.json  accuracy   -         0.947        -
ml-example3\evaluation-result\metrics.json  f1_score   -         0.8656       -
ml-example3\evaluation-result\metrics.json  precision  -         0.988        -
ml-example3\evaluation-result\metrics.json  recall     -         0.7702       -


### Commiting changes in myfeature branch

In [10]:
!git branch
!git checkout myfeature

  master
* myfeature
M	Track4-MLTrainning.ipynb


Already on 'myfeature'


In [11]:
!dvc push
!git add .
!git commit -m "DVC Pipeline for ml-example3"
!git push origin myfeature

Everything is up to date.


[myfeature 0261c0e] DVC Pipeline for ml-example3
 1 file changed, 15 insertions(+), 41 deletions(-)


To https://github.com/jacesca/CICD-Workflow.git
   f91ee65..0261c0e  myfeature -> myfeature


### Setting up DVC Github Action

```
name: comments-ml-example3

on:
  pull_request:
    branches: ["master", "myfeature"]

permissions: write-all

jobs:
  train_and_test_model_ml2:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout
        uses: actions/checkout@v4

      - name: Setup Python
        uses: actions/setup-python@v5
        with:
          python-version: 3.11.9

      - name: Install Dependencies
        run: |
          pip install -r requirements.txt

      - name: Train and test model
        run: |
          python ml-example2/preprocess_data.py
          python ml-example2/train.py

      - name: Setup DVC
        uses: iterative/setup-dvc@v1

      - name: Run DVC pipeline
        run: dvc repro
        
      - name: Setup CML GitHub Actions
        uses: iterative/setup-cml@v3

      - name: Write CML report
        env:
          REPO_TOKEN: ${{secrets.GITHUB_TOKEN}}
        run: |
          echo "# ML Example 3" > report3.md
          dvc metrics show --md >> report3.md
          echo "### Metrics comparison with master branch"
          dvc metrics diff --all --md >> report3.md
          echo "![Confusion Matrix Plot](ml-example3/evaluation-result/confusion_matrix.png)" >> report3.md
          cml comment create report3.md
```

### Plotting Confusion Matrix

In [30]:
# Plotting what was configured
!dvc plots show

file:///C:/Users/Jacqueline/Documents/projects/CAMP-MLEngTrack/14-CICD%20for%20Machine%20Learning/dvc_plots/index.html


In [31]:
from IPython.display import IFrame
IFrame("dvc_plots/index.html", width=800, height=450)

In [49]:
# Plotting other graphs
# scatter - scatter plot
# linear - interactive linear plot
# simple - non-interactive customizable linear plot
# smooth - linear plot with smoothing
# confusion - confusion matrix
# confusion_normalized - confusion matrix with values normalized to <0, 1> range
# bar_horizontal - horizontal bar plot
# bar_horizontal_sorted - horizontal bar plot sorted by bar size
!dvc plots show ml-example3/evaluation-result/predictions.csv \
                -t confusion \
                -x predicted_label \
                -y true_label \
                --x-label "Predicted Label" \
                --y-label "True label" \
                --title "Confusion matrix"

file:///C:/Users/Jacqueline/Documents/projects/CAMP-MLEngTrack/14-CICD%20for%20Machine%20Learning/dvc_plots/index.html


In [50]:
from IPython.display import IFrame
IFrame("dvc_plots/index.html", width=800, height=450)

### Comparing Confusion Matrix

In [51]:
!dvc plots diff --all

usage: dvc plots diff [-h] [-q | -v] [--targets [<paths> ...]] [-t [<path>]]
                      [-x <field>] [-y <field>] [--no-header] [--title <text>]
                      [--x-label <text>] [--y-label <text>] [-o <path>]
                      [--show-vega] [--open] [--html-template <path>]
                      [revisions ...]

Show multiple versions of a plot by overlaying them in a single image.
Documentation: <https://man.dvc.org/plots/diff>

positional arguments:
  revisions             Git commits to plot from

options:
  -h, --help            show this help message and exit
  -q, --quiet           Be quiet.
  -v, --verbose         Be verbose.
  --targets [<paths> ...]
                        Specific plots to visualize. Accepts any file path or
                        plot name from `dvc.yaml` file. Shows all tracked
                        plots by default.
  -t [<path>], --template [<path>]
                        Special JSON or HTML schema file to inject with the
     

ERROR: unrecognized arguments: --all


### Ex.1 - Adding metrics and plots to dvc.yaml
In this exercise, your task is to complete the contents of dvc.yaml that defines a model training workflow.

Here preprocess_dataset.py and train.py are the files that perform data preprocessing and model training by taking weather.csv as input in the raw_dataset folder. As output, the model training code generates a predictions.csv file that contains the predictions and the ground truth, and metrics.json file containing structured metrics data. The former would be used to generate a normalized confusion matrix plot for comparing it with previous commits.

**Instruction**
1. Set the metrics target to the output metrics file.
2. Set the plot target to the output file containing predictions data.
3. Set the plot template to confusion_normalized to plot the normalized confusion matrix.
4. Set the correct value for cache key to track plots in Git repository instead of DVC remote.

In [12]:
!more dvc.yaml


stages:
  preprocess:
    cmd: python ml-example3/preprocess_data.py
    deps:
    - ml-example3/raw-data/weather.csv
    - ml-example3/preprocess_data.py
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/processed-data/weather.csv
  train:
    cmd: python ml-example3/train.py
    deps:
    - ml-example3/metrics_and_plots.py
    - ml-example3/model.py
    - ml-example3/processed-data/weather.csv
    - ml-example3/train.py
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/evaluation-result/confusion_matrix.png
    metrics:
      - ml-example3/evaluation-result/metrics.json:
          cache: false
    plots:
      - ml-example3/evaluation-result/predictions.csv:
          template: confusion_normalized
          x: predicted_label
          y: true_label
          x_label: 'Predicted label'
          y_label: 'True label'
          title: Confusion matrix
          cache: false
